In [28]:
import rawpy
import cv2
import numpy as np



In [29]:
#Read raw images
path='Images/Original.ARW'
raw=rawpy.imread(path)


#Postprocess the currently loaded RAW image and return the new resulting image as numpy array.

rgb_img=raw.postprocess()


#Changing the colorspace from RGB to YCbCr
ycc_img=cv2.cvtColor(rgb_img,cv2.COLOR_XYZ2RGB)

#Center
ycc_img=ycc_img.astype(int)-128

In [30]:

def downsampling(x,ratio='4:2:0'):
    assert ratio in ('4:4:0', '4:2:2','4:2:0'),"{'4:4:4', '4:2:2', '4:2:0'}"
    
    #No subsampling
    
    if ratio=='4:4:4':
        return x
    
    else:
        out=np.zeros((x.shape))
        
        #Downsampling in the window of 2 in the horizontal direction
        
        if ratio=='4:4:2':
            for i in range(0,x.shape[0],2):
                out[i:i+2]=np.mean(x[i:i+2],axis=0)
                
        #Downsample with a window of 2 in both directions
        
        else:
            for i in range(0,x.shape[0],2):
                for j in range(0,x.shape[1],2):
                    out[i:i+2, j:j+2]=np.mean(x[i:i+2,j:j+2])
        
    return np.round(out).astype('uint8')


                

In [31]:
lum_downsample=downsampling(ycc_img,ratio='4:4:0')
chr_downsample=downsampling(ycc_img,ratio='4:2:0')

Y=lum_downsample(ycc_img[:,:,0]) #Luma
Cr=chr_downsample(ycc_img[:,:,1]) #Red Chroma
Cb=chr_downsample(ycc_img[:,:,2]) # Blue Chroma

ycc_img=np.stack((Y,Cr,Cb),axis=2) #Stack the channels back

TypeError: 'numpy.ndarray' object is not callable

In [ ]:
class ImageBlock():
    def __init__(self, block_height=8, block_width=8):
        self.block_height = block_height
        self.block_width = block_width
        self.left_padding = self.right_padding = self.top_padding = self.bottom_padding = 0
    
    def forward(self, image):
        self.image_height = image.shape[0]
        self.image_width = image.shape[1]
        self.image_channel = image.shape[2]
    
        # Vertical padding
        if self.image_height % self.block_height != 0:
            vpad = self.image_height % self.block_height
            self.top_padding = vpad // 2 
            self.bottom_padding = vpad - self.top_padding
            image = np.concatenate((np.repeat(image[:1], self.top_padding, 0), image, 
                                    np.repeat(image[-1:], self.bottom_padding, 0)), axis=0)
            
        # Horizontal padding
        if self.image_width % self.block_width != 0:
            hpad = self.image_width % self.block_width
            self.left_padding = hpad // 2 
            self.right_padding = hpad - self.left_padding
            image = np.concatenate((np.repeat(image[:,:1], self.left_padding, 1), image, 
                                    np.repeat(image[:,-1:], self.right_padding, 1)), axis=1)
    
        # Update dimension
        self.image_height = image.shape[0]
        self.image_width = image.shape[1]

        # Create blocks
        blocks = []
        indices = []
        for i in range(0, self.image_height, self.block_height):
            for j in range(0, self.image_width, self.block_width):
                for k in range(self.image_channel):
                    blocks.append(image[i:i+self.block_height, j:j+self.block_width, k])
                    indices.append((i,j,k))
                    
        blocks = np.array(blocks)
        indices = np.array(indices)
        return blocks, indices
    
    def backward(self, blocks, indices):
        # Empty image array
        image = np.zeros((self.image_height, self.image_width, self.image_channel)).astype(int)
        for block, index in zip(blocks, indices):
            i, j, k = index
            image[i:i+self.block_height, j:j+self.block_width, k] = block
            
        # Remove padding
        if self.top_padding > 0:
            image = image[self.top_padding:,:,:]
        if self.bottom_padding > 0:
            image = image[:-self.bottom_padding,:,:] 
        if self.left_padding > 0:
            image = image[:,self.left_padding:,:]
        if self.right_padding > 0:
            image = image[:,:-self.right_padding,:]
        return image

In [ ]:
from scipy.fft import dct

def dct2d(x):
    out=dct(dct(x,norm=))